In [206]:
import sys
import doctest
import random
import math
import itertools as it
import pathlib
from typing import List, Tuple

import bs4
import urllib

def get_title(day):
    f = urllib.request.urlopen(f"https://adventofcode.com/2019/day/{day}")
    soup = bs4.BeautifulSoup(f.read())
    print(soup.h2.text.replace("-","").strip())
    
def get_path(day):
    return pathlib.Path(f"./2019/day{day:02}.txt")

## Day 1: The Tyranny of the Rocket Equation

In [71]:
def day01_read():
    file_ = get_path(1)
    return [int(line) for line in open(file_).readlines()]
    

def day01(mass: int) -> int:
    """
    >>> day01(12)
    2
    >>> day01(14)
    2
    >>> day01(1969)
    654
    >>> day01(100756)
    33583
    """
    return mass // 3 - 2


In [72]:
inputs = day01_read()
output = sum(day01(x) for x in inputs)
output

3286680

In [74]:
def day01_mod(mass: int) -> int:
    """
    >>> day01_mod(14)
    2
    >>> day01_mod(1969)
    966
    >>> day01_mod(100756)
    50346
    """
    res = []
    while True:
        mass = day01(mass)
        if mass > 0:
            res.append(mass)
        else:
            break
    return sum(res)

In [75]:
doctest.testmod()

TestResults(failed=0, attempted=7)

In [76]:
inputs = day01_read()
output = sum(day01_mod(x) for x in inputs)
output

4927158

## Day 2: 1202 Program Alarm

In [77]:
def day02_read():
    file_ = get_path(2)
    return [int(x) for x in open(file_).read().strip().split(",")]
   

def day02(state: List[int]) -> int:
    """
    >>> day02([1,9,10,3,2,3,11,0,99,30,40,50])
    3500
    >>> day02([1,0,0,0,99])
    2
    >>> day02([2,4,4,5,99,0])
    2
    >>> day02([1,1,1,4,99,5,6,0,99])
    30
    """
    def _op1(xs: List[int], loc: int) -> List[int]:
        xs[xs[loc + 3]] = xs[xs[loc + 1]] + xs[xs[loc + 2]]
        return xs, loc + 4


    def _op2(xs: List[int], loc: int) -> List[int]:
        xs[xs[loc + 3]] = xs[xs[loc + 1]] * xs[xs[loc + 2]]
        return xs, loc + 4
    
    loc = 0
    while state[loc] != 99:
        if state[loc] == 1:
            state, loc = _op1(state, loc)
        elif state[loc] == 2:
            state, loc = _op2(state, loc)
    return state[0]

In [78]:
doctest.testmod()

TestResults(failed=0, attempted=11)

In [79]:
inputs = day02_read()
inputs[1] = 12
inputs[2] = 2
day02(inputs[:])

9706670

In [80]:
def day02_mod(xs: List[int], target: int) -> int:
    size = len(xs)
    for noun in range(size):
        for verb in range(size):
            ys = xs[:]
            ys[1] = noun
            ys[2] = verb
            if day02(ys) == target:
                return 100 * noun + verb

In [81]:
day02_mod(inputs, 19690720)

2552

## Day 3: Crossed Wires

### Part 1

In [82]:
def day03_read():
    file_ = get_path(3)
    ss = open(file_).readlines()
    res = [s.strip() for s in ss]
    return res


def _day03_exec_command(cmd, state):
    dir_, steps = cmd[0], int(cmd[1:])
    coord, record = state
    x, y = coord
    if dir_ == "R":
        dx, dy = 1, 0
    elif dir_ == "L":
        dx, dy = -1, 0
    elif dir_ == "U":
        dx, dy = 0, 1
    else:
        assert dir_ == "D"
        dx, dy = 0, -1
        
    for i in range(1, steps + 1):
        record.add((x + dx * i, y + dy * i))
                    
    coord = (x + dx * steps, y + dy * steps)
    return coord, record


def day03(commands1, commands2):
    """
    >>> day03("R8,U5,L5,D3", "U7,R6,D4,L4")
    6
    
    >>> day03("R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83")
    159
    
    >>> day03("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
    135
    
    """

    def _run(commands):
        commands = commands.split(",")
        state = (0, 0), set()
        for cmd in commands:
            state = _day03_exec_command(cmd, state)
        _, record = state
        return record

    rec1 = _run(commands1)
    rec2 = _run(commands2)
    crossings = rec1 & rec2
    nearest = min(sum(map(abs, point)) for point in crossings)
    return nearest

In [83]:
doctest.testmod()

TestResults(failed=0, attempted=14)

In [85]:
command1, commands2 = day03_read()
day03(command1, commands2)

651

### Part 2

There is ambiguity of step count when a trajectory crosses by itself. Assume that such self-crossing points do not appear as crossing points between trajectories.

In [86]:
def _day03_exec_command_mod(cmd, state):
    coord, record = state
    steps = record[coord]
    dir_, stride = cmd[0], int(cmd[1:])
    x, y = coord
    if dir_ == "R":
        dx, dy = 1, 0
    elif dir_ == "L":
        dx, dy = -1, 0
    elif dir_ == "U":
        dx, dy = 0, 1
    else:
        assert dir_ == "D"
        dx, dy = 0, -1
        
    for i in range(1, stride + 1):
        coord = (x + dx * i, y + dy * i)
#         if coord in record:
#             print("Already visited!")
        record[coord] = steps + i
                    
    coord = (x + dx * stride, y + dy * stride)
    return coord, record


def _day03_process_mod(commands):
    commands = commands.split(",")
    state = (0, 0), {(0, 0): 0}
    for cmd in commands:
        state = _day03_exec_command_mod(cmd, state)
    _, record = state
    return record


def day03_mod(commands1, commands2):
    """
    >>> day03_mod("R8,U5,L5,D3", "U7,R6,D4,L4")
    30
    
    >>> day03_mod("R75,D30,R83,U83,L12,D49,R71,U7,L72", "U62,R66,U55,R34,D71,R55,D58,R83")
    610
    
    >>> day03_mod("R98,U47,R26,D63,R33,U87,L62,D20,R33,U53,R51", "U98,R91,D20,R16,D67,R40,U7,R15,U6,R7")
    410
    """
    rec1 = _day03_process_mod(commands1)
    rec2 = _day03_process_mod(commands2)
    crossings = (set(rec1.keys()) & set(rec2.keys())) - {(0, 0)}
    nearest = min(rec1[point] + rec2[point] for point in crossings)
    return nearest

In [122]:
doctest.testmod()

TestResults(failed=0, attempted=25)

In [87]:
command1, commands2 = day03_read()
day03_mod(command1, commands2)

7534

In [88]:
get_title(4)

Day 4: Secure Container


## Day 4: Secure Container


Wasted a lot of time in interpreting the problem correctly.

In [95]:
def day04(s):
    from_, to_ = map(int, s.split("-"))
    from_ = max(100000, from_)
    to_ = min(999999, to_)
    res = sum(1 for i in range(from_, to_ + 1)
              if (all(c1 <= c2 for c1, c2 in zip(str(i), str(i)[1:])) and
                  any(c1 == c2 for c1, c2 in zip(str(i), str(i)[1:])))
          )
    return res

In [96]:
inputs = "138307-654504"
day04(inputs)

1855

In [97]:
def day04_mod(s):
    from_, to_ = map(int, s.split("-"))
    from_ = max(100000, from_)
    to_ = min(999999, to_)
    
    res = 0
    for i in range(from_, to_ + 1):
        ss = str(i)
        
        if all(c1 <= c2 for c1, c2 in zip(ss, ss[1:])):
            cnt = collections.Counter(ss)
            if any(x for x in cnt.values() if x == 2):
                res += 1
    return res

In [98]:
day04_mod(inputs)

1253

In [21]:
get_title(5)

Day 5: Sunny with a Chance of Asteroids


## Day 5: Sunny with a Chance of Asteroids

### Part 1 & Part 2

I mean a `state` by a snapshot of time-chaninging intcode. 

In [121]:
def day05_read():
    file_ = get_path(5)
    return [int(x) for x in open(file_).read().strip().split(",")]


def day05(state: List[int], input_: int, verbose=False) -> int:
    """
    >>> day05([3,9,8,9,10,9,4,9,99,-1,8], 8)
    [1]
    >>> day05([3,9,8,9,10,9,4,9,99,-1,8], -3)
    [0]
    >>> day05([3,9,7,9,10,9,4,9,99,-1,8], 4)
    [1]
    >>> day05([3,9,7,9,10,9,4,9,99,-1,8], 8)
    [0]
    >>> day05([3,3,1108,-1,8,3,4,3,99], 8)
    [1]
    >>> day05([3,3,1108,-1,8,3,4,3,99], 7)
    [0]
    >>> day05([3,3,1107,-1,8,3,4,3,99], 1)
    [1]
    >>> day05([3,3,1107,-1,8,3,4,3,99], 9)
    [0]
    """
    res = []
    
    def _decode_opcode(n:int) -> Tuple[int, bool, bool, bool]:
        op = n % 100       # take two right-most digits as int
        s = str(n).zfill(5)
        mode3 = s[0] == '0'  # True iff position mode
        mode2 = s[1] == '0'  # False iff immediate mode
        mode1 = s[2] == '0'
        return op, mode1, mode2, mode3

    def _bin_op(f, xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        left = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        right = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        if mode[2]:
            xs[xs[loc + 3]] = f(left, right)
        else:
            raise ValueError("Something is wrong")
        return xs, loc + 4

    def _add(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: a + b, xs, loc, *mode)
    
    def _mul(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: a * b, xs, loc, *mode)

    def _in(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        xs[xs[loc + 1]] = input_
        return xs, loc + 2

    def _out(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        out = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        res.append(out)
        return xs, loc + 2

    def _jmp_true(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        x = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        if x > 0:
            loc = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        else:
            loc += 3
        return xs, loc

    def _jmp_false(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        x = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        if x == 0:
            loc = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        else:
            loc += 3
        return xs, loc

    def _lt(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: int(a < b), xs, loc, *mode)
    
    def _eq(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: int(a == b), xs, loc, *mode)    

    opfun = {1: _add, 2: _mul, 3: _in, 4: _out, 5:_jmp_true, 6:_jmp_false, 7: _lt, 8: _eq}
    
    loc = 0
    while state[loc] != 99:
        if verbose:
            print(f"processing {state[loc]} at {loc}", file=sys.stderr)
        opcode = state[loc]
        op, mode1, mode2, mode3 = _decode_opcode(opcode)
        state, loc = opfun[op](state, loc, mode1, mode2, mode3)
    return res

In [122]:
doctest.testmod()

TestResults(failed=0, attempted=25)

In [123]:
def test_day05():
    xs = [3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99]
    x = random.randrange(20)
    expected = 999 if x < 8 else 1000 if x == 8 else 1001
    assert day05(xs, x) == [expected]

In [124]:
test_day05()

In [125]:
inputs = day05_read()
inputs[:10]

[3, 225, 1, 225, 6, 6, 1100, 1, 238, 225]

In [126]:
day05(inputs[:], 1)

[0, 0, 0, 0, 0, 0, 0, 0, 0, 4887191]

In [127]:
day05(inputs[:], 5)

[3419022]

In [128]:
get_title(6)

Day 6: Universal Orbit Map


## Day 6: Universal Orbit Map

### Part 1

A DAG with the unique minimum node. Find # of steps from a node to the minimum node.

In [129]:
def day06_read():
    file_ = get_path(6)
    return [line.strip() for line in open(file_).readlines() if line.strip()]

In [130]:
inputs = day06_read()
inputs[:3]

['WR4)TZN', 'M6J)Q1N', 'D5B)2KR']

In [131]:
def _day06_to_adj(lines):
    d = collections.defaultdict(list)
    for line in lines:
        inner, outer = line.split(")")
        d[outer].append(inner)
    
    assert(all(len(lis) == 1 for lis in d.values()))
    return {k:v[0] for k,v in d.items()}

In [132]:
adj = to_adj(inputs)
adj

{'TZN': 'WR4',
 'Q1N': 'M6J',
 '2KR': 'D5B',
 '8JX': 'ZNY',
 'N48': '349',
 'GTH': '2RY',
 'SV6': '84X',
 '8XL': 'F26',
 'BV8': 'QT8',
 'Z91': 'Q7X',
 '5YM': 'CHD',
 'QLS': '3F1',
 'WSB': 'M9P',
 '6J7': 'B1D',
 'YDQ': '4CG',
 '23H': 'ZYW',
 'KGS': 'BK9',
 '8PF': 'NHH',
 'TT8': '53H',
 'T72': 'P5X',
 'S52': '54T',
 'SM2': 'FC1',
 'H2L': 'M82',
 '73K': '4SF',
 'T5V': '7QG',
 '1HW': '7NH',
 'PPD': 'FCQ',
 '5VM': 'NWP',
 '861': 'TMX',
 'Z9N': '358',
 'XP8': 'ZSC',
 '91X': '8M9',
 'QJM': '8PF',
 'S5Y': 'ZQJ',
 'ZBQ': 'H4C',
 '4XN': 'KZ8',
 'H4C': '2L8',
 'BNB': 'DP6',
 '2HB': 'XTZ',
 '61P': 'X4X',
 'LKS': '2NL',
 'X27': 'R4L',
 'S8D': 'GWG',
 'B7H': '32W',
 '5QF': 'HDC',
 'Y8M': 'B52',
 'N57': 'CQ8',
 'W8T': 'PSY',
 '592': '43M',
 'BZK': 'H6Z',
 'NFG': 'N6M',
 'YKZ': '58M',
 '9CG': 'TCL',
 'Y6S': '855',
 'BCF': 'M7N',
 '5HK': 'GLC',
 'WT7': 'MDY',
 'ZHL': 'LPD',
 '94F': 'L87',
 'TCV': 'KHH',
 'D7T': 'XKM',
 '2KD': 'WX6',
 'DPP': '4DM',
 'Q55': 'XQD',
 'D1Y': 'KSC',
 'MMB': '168',
 'W6B': 'P

In [133]:
def day06(lines):
    adj = _day06_to_adj(lines)
    def _count(k):
        res = 0
        while k != "COM":
            k = adj[k]
            res += 1
        return res
    return sum(_count(k) for k in adj.keys())

In [134]:
day06(inputs)

160040

### Part 2
Find the minimum sum of "steps" from two nodes ("YOU" and "SAN") to their common descendant node.

In [135]:
test = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L""".split()
day06(test)

42

In [136]:
def day06_mod(lines):
    adj = _day06_to_adj(lines)
    def _traj():
        k = "SAN"
        res = []
        while k != "COM":
            k = adj[k]
            res.append(k)
        return res
    
    traj = _traj()
    traj_set = set(traj)
    k = "YOU"
    res = 0
    while k not in traj_set:
        k = adj[k]
        res += 1
        
    idx = traj.index(k)
    return res + idx - 1

In [137]:
test = """COM)B
B)C
C)D
D)E
E)F
B)G
G)H
D)I
E)J
J)K
K)L
K)YOU
I)SAN""".split()
day06_mod(test)

4

In [138]:
day06_mod(inputs)

373

In [139]:
get_title(7)

Day 7: Amplification Circuit


## Day 7: Amplification Circuit

### Part 1 
Continuation to INTCODE computer from day 5.

In [10]:
def day07_read():
    file_ = get_path(7)
    assert file_.exists()
    return [int(w) for w in open(file_).read().split(',')]

In [11]:
inputs = day07_read()
orig = inputs[:]
inputs[:10]

[3, 8, 1001, 8, 10, 8, 105, 1, 0, 0]

In [62]:
def intcomputer(intcode, phase, inputs_rest, verbose=True):
    """
    """
    res = []
    input_iter = iter(it.chain([phase], inputs_rest))
    
    def _decode_opcode(n:int) -> Tuple[int, bool, bool, bool]:
        op = n % 100       # take two right-most digits as int
        s = str(n).zfill(5)
        mode3 = s[0] == '0'  # True iff position mode
        mode2 = s[1] == '0'  # False iff immediate mode
        mode1 = s[2] == '0'
        return op, mode1, mode2, mode3

    def _bin_op(f, xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        left = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        right = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        if mode[2]:
            xs[xs[loc + 3]] = f(left, right)
        else:
            raise ValueError("Something is wrong")
        return xs, loc + 4

    def _add(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: a + b, xs, loc, *mode)
    
    def _mul(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: a * b, xs, loc, *mode)

    def _in(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        xs[xs[loc + 1]] = next(input_iter)
        return xs, loc + 2

    def _out(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        out = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        res.append(out)
        return xs, loc + 2

    def _jmp_true(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        x = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        if x > 0:
            loc = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        else:
            loc += 3
        return xs, loc

    def _jmp_false(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        x = xs[xs[loc + 1]] if mode[0] else xs[loc + 1]
        if x == 0:
            loc = xs[xs[loc + 2]] if mode[1] else xs[loc + 2]
        else:
            loc += 3
        return xs, loc

    def _lt(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: int(a < b), xs, loc, *mode)
    
    def _eq(xs: List[int], loc: int, *mode) -> Tuple[List[int], int]:
        return _bin_op(lambda a, b: int(a == b), xs, loc, *mode)    

    opfun = {1: _add, 2: _mul, 3: _in, 4: _out, 5:_jmp_true, 6:_jmp_false, 7: _lt, 8: _eq}
    
    loc = 0
    while intcode[loc] != 99:
        if verbose:
            print(f"processing {intcode[loc]} at {loc}", file=sys.stderr)
        opcode = intcode[loc]
        op, mode1, mode2, mode3 = _decode_opcode(opcode)
        intcode, loc = opfun[op](intcode, loc, mode1, mode2, mode3)
    return res

In [65]:
def day07_amp(intcode, phases, verbose=False):
    """
    >>> day07_amp([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0], [4,3,2,1,0])
    [43210]
    >>> day07_amp([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0], [0,1,2,3,4])
    [54321]
    >>> day07_amp([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0], [1,0,4,3,2])
    [65210]
    """
    if isinstance(phases, int):
        phases = str(phases)
    if isinstance(phases, str):
        phases = list(map(int, phases))
        
    out = it.repeat(0)
    for phase in phases:
        if verbose:
            print(f"phase = {phase}", file=sys.stderr)
        out = intcomputer(intcode, phase, out, verbose=verbose)
    return out

In [66]:
doctest.testmod()

**********************************************************************
File "__main__", line 3, in __main__.day07_amp_feedback
Failed example:
    day07_amp_feedback([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5])
Expected:
    [139629729]
Got:
    [5, 5, 5, 5, 5]
    [14]
    [31]
    [64]
    [129]
    [263]
    [530]
    [1063]
    [2128]
    [4257]
    [8519]
    [17042]
    [34087]
    [68176]
    [136353]
    [136353]
**********************************************************************
1 items had failures:
   1 of   1 in __main__.day07_amp_feedback
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=7)

In [67]:
def day07(intcode):
    """
    >>> day07([3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0])
    43210
    >>> day07([3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0])
    54321
    >>> day07([3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,31,1,32,31,31,4,31,99,0,0,0])
    65210
    """
    max_signal = max(day07_amp(intcode, phases)[0] 
                     for phases in it.permutations(range(5)))
    return max_signal

In [68]:
doctest.testmod()

**********************************************************************
File "__main__", line 3, in __main__.day07_amp_feedback
Failed example:
    day07_amp_feedback([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5])
Expected:
    [139629729]
Got:
    [5, 5, 5, 5, 5]
    [14]
    [31]
    [64]
    [129]
    [263]
    [530]
    [1063]
    [2128]
    [4257]
    [8519]
    [17042]
    [34087]
    [68176]
    [136353]
    [136353]
**********************************************************************
1 items had failures:
   1 of   1 in __main__.day07_amp_feedback
***Test Failed*** 1 failures.


TestResults(failed=1, attempted=7)

In [69]:
day07(inputs)

38834

In [22]:
orig[:10]

[3, 8, 1001, 8, 10, 8, 105, 1, 0, 0]

### Part 2

In [102]:
def day07_amp_feedback(intcode, phases, verbose=False):
    """
    >>> day07_amp_feedback([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5])
    [139629729]
    """
    if isinstance(phases, int):
        phases = str(phases)
    if isinstance(phases, str):
        phases = list(map(int, phases))
        
    out = [0] * 10
    intcodes = [intcode[:] for _ in range(5)]
    for _ in range(10):
        for i, phase in enumerate(phases):
            if verbose:
                print(f"phase = {phase}", file=sys.stderr)
                print(f"out = {out}", file=sys.stderr)
            out = intcomputer(intcodes[i], phase, out, verbose=verbose)
            print(out)
    return out

In [103]:
xs = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
xss = [xs[:] for _ in range(5)]
intcomputer(xss[0], 9, [0]*5, True)

processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22


[5, 5, 5, 5, 5]

In [91]:
intcomputer(xss[0], 9, [129]*5, True)

processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22


[263]

In [104]:
day07_amp_feedback(xs[:], [9,8,7,6,5])

[5, 5, 5, 5, 5]
[14, 14, 14, 14, 14]
[31, 31, 31, 31, 31]
[64, 64, 64, 64, 64]
[129, 129, 129, 129, 129]
[263]
[530]
[1063]
[2128]
[4257]
[8519]
[17042]
[34087]
[68176]
[136353]
[272711]
[545426]
[1090855]
[2181712]
[4363425]
[8726855]
[17453714]
[34907431]
[69814864]
[139629729]
[279259463]
[558518930]
[1117037863]
[2234075728]
[4468151457]
[8936302919]
[17872605842]
[35745211687]
[71490423376]
[142980846753]
[285961693511]
[571923387026]
[1143846774055]
[2287693548112]
[4575387096225]
[9150774192455]
[18301548384914]
[36603096769831]
[73206193539664]
[146412387079329]
[292824774158663]
[585649548317330]
[1171299096634663]
[2342598193269328]
[4685196386538657]


[4685196386538657]

In [87]:
intcomputer(ys, 9, [0]*10)

processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1 at 8
processing 1007 at 12
processing 1005 at 16
processing 1001 at 19
processing 1105 at 23
processing 1007 at 12
processing 1005 at 16
processing 1001 at 19
processing 1105 at 23
processing 1007 at 12
processing 1005 at 16
processing 1 at 26
processing 1008 at 30
processing 1001 at 34
processing 2 at 38
processing 4 at 42
processing 1001 at 44
processing 1005 at 48
processing 3 at 6
processing 1 at 8
processing 1007 at 12
processing 1005 at 16
processing 1001 at 19
processing 1105 at 23
processing 1007 at 12
processing 1005 at 16
processing 1001 at 19
processing 1105 at 23
processing 1007 at 12
processing 1005 at 16
processing 1 at 26
processing 1008 at 30
processing 1001 at 34
processing 2 at 38
processing 4 at 42
processing 1001 at 44
processing 1005 at 48
processing 3 at 6
processing 1 at 8
processing 1007 at 12
processing 1005 at 16
processing 1001 at 19
processing 1105 at 23
processing 1007 at 12
processing 10

[4, 3, 2, 1, 0, 4, 3, 2, 1, 0]

In [88]:
intcomputer(ys, 9, [19, 20, 12, 15, 10, 19, 20, 12, 15, 10])

processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1 at 8
processing 1007 at 12
processing 1005 at 16
processing 1 at 26
processing 1008 at 30
processing 1001 at 34
processing 2 at 38
processing 4 at 42
processing 1001 at 44
processing 1005 at 48


[23]

In [105]:
ys = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
      -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
      53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
day07_amp_feedback(ys[:], [9,7,8,5,6])

[4, 3, 2, 1, 0, 4, 3, 2, 1, 0]
[6, 4, 4, 5, 3, 6, 4, 4, 5, 3]
[9, 6, 5, 10, 7, 9, 6, 5, 10, 7]
[18, 10, 8, 12, 8, 18, 10, 8, 12, 8]
[19, 20, 12, 15, 10, 19, 20, 12, 15, 10]
[23]
[25]
[28]
[56]
[57]
[60]
[61]
[63]
[62]
[124]
[126]
[252]
[253]
[251]
[250]
[251]
[250]
[500]
[497]
[495]
[990]
[988]
[987]
[983]
[980]
[979]
[976]
[974]
[969]
[965]
[963]
[959]
[956]
[950]
[945]
[942]
[937]
[933]
[926]
[920]
[916]
[910]
[905]
[897]
[890]


[890]

In [108]:
# def day07_mod(intcode):
#     """
#     >>> day07_mod([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])
#     139629729
#     >>> day07_mod([3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,-5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10])
#     18216
#     """
#     max_signal = max(day07_amp(intcode, phases)[0]
#                      for phases in it.permutations(range(5, 10)))
#     return max_signal

In [106]:
day07_amp([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5], [9,8,7,6,5], verbose=True)

phase = 9
processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
phase = 8
processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
phase = 7
processing 3 at 0
processing 1001 at 2
processing 3 at 6
processing 1002 at 8
processing 1 at 12
processing 4 at 16
processing 1001 at 18
processing 1005 at 22
phase 

[129]

In [109]:
get_title(8)

Day 8: Space Image Format


In [230]:
doctest.testmaod()

**********************************************************************
File "__main__", line 3, in __main__.day07_mod
Failed example:
    day07_mod([3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5])
Exception raised:
    Traceback (most recent call last):
      File "<ipython-input-229-e26d247d6990>", line 9, in <genexpr>
        for phases in it.permutations(range(5, 10)))
      File "<ipython-input-221-0c674c85b339>", line 17, in day07_amp
        out = intcomputer(intcode, [phase, out], verbose=False)
      File "<ipython-input-220-51fdd08d1ef2>", line 69, in intcomputer
        intcode, loc = opfun[op](intcode, loc, mode1, mode2, mode3)
      File "<ipython-input-220-51fdd08d1ef2>", line 31, in _in
        xs[xs[loc + 1]] = next(input_iter)
    StopIteration

    The above exception was the direct cause of the following exception:

    Traceback (most recent call last):
      File "/Users/yamato/miniconda3/lib/python3.7/doctest.py", line 1329, 

TestResults(failed=2, attempted=33)

## Day 8: Space Image Format

### Part 1

In [113]:
def day08_read():
    p = get_path(8)
    return open(p).read().strip()

In [115]:
inputs = day08_read()
inputs[:100]

'2222222222222202221222222200222222222222022222222222222202022222222222012122221202222222202221022222'

In [117]:
len(inputs)

15000

In [122]:
# https://docs.python.org/3/library/itertools.html
def grouper(iterable, n, fillvalue=None):
    "Collect data into fixed-length chunks or blocks"
    # grouper('ABCDEFG', 3, 'x') --> ABC DEF Gxx"
    args = [iter(iterable)] * n
    return it.zip_longest(*args, fillvalue=fillvalue)

In [123]:
def day08(s):
    chunks = grouper(s, 25 * 6)
    ss = min(chunks, key=lambda x: collections.Counter(x)['0'])
    cnt = collections.Counter(ss)
    return cnt['1'] * cnt['2']

In [124]:
day08(inputs)

1965

### Part 2

In [155]:
def _day08_numpy(s):
    xs = np.array([int(c) for c in s])
    return xs.reshape(-1, 6, 25)

def _day08_show(xss):
    s = "\n".join("".join(map(lambda x: "X" if x == 0 else ' ', xs)) for xs in xss)
    print(s)
    
def _day08_reduce(xs):
    return next(it.dropwhile(lambda x: x == 2, xs))

def day08_mod(s):
    xss = _day08_numpy(s)
    _day08_show(np.apply_along_axis(_day08_reduce, 0, xss))

In [158]:
day08_mod(inputs)

X  XX    X XX XXX  X XXX 
 XX XXXX X X XXXXX X XXX 
 XXXXXX XX  XXXXXX XX X X
 X  XX XXX X XXXXX XXX XX
 XX X XXXX X XX XX XXX XX
X   X    X XX XX  XXXX XX


In [159]:
get_title(9)

Day 9: Sensor Boost


## Day 9: Sensor Boost

In [160]:
get_title(10)

Day 10: Monitoring Station


## Day 10: Monitoring Station

### Part 1

In [161]:
import fractions

In [164]:
def day10_read():
    p = get_path(10)
    return [line.strip() for line in open(p).readlines()]

In [196]:
inputs = day10_read()
inputs

['.###.###.###.#####.#',
 '#####.##.###..###..#',
 '.#...####.###.######',
 '######.###.####.####',
 '#####..###..########',
 '#.##.###########.#.#',
 '##.###.######..#.#.#',
 '.#.##.###.#.####.###',
 '##..#.#.##.#########',
 '###.#######.###..##.',
 '###.###.##.##..####.',
 '.##.####.##########.',
 '#######.##.###.#####',
 '#####.##..####.#####',
 '##.#.#####.##.#.#..#',
 '###########.#######.',
 '#.##..#####.#####..#',
 '#####..#####.###.###',
 '####.#.############.',
 '####.#.#.##########.']

In [228]:
def _to_int_frac_pair(dx, dy):
    if dx == 0:
        return (0, dy // abs(dy))
    
    return (dx // abs(dx), fractions.Fraction(dy, abs(dx)))

def _count(positions, p):
    px, py = p
    res = set()
    for q in positions:
        if q == p:
            continue
        qx, qy = q
        pair = _to_int_frac_pair(qx - px, qy - py)
        res.add(pair)
    return len(res)

def _get_asteroid_locs(grid):
    width = len(grid[0])
    height = len(grid)
    res = {
        (j, i)  # record as (x, y) defined in the problem
        for i in range(height)
        for j in range(width)
        if grid[i][j] == "#"
    }
    return res

def day10(grid):
    asteroid_locs = _get_asteroid_locs(grid)
    return max((_count(asteroid_locs, p), p) for p in asteroid_locs)

In [229]:
test1 = """
.#..#
.....
#####
....#
...##""".strip().split("\n")

day10(test1)

(8, (3, 4))

In [230]:
test2 = """
......#.#.
#..#.#....
..#######.
.#.#.###..
.#..#.....
..#....#.#
#..#....#.
.##.#..###
##...#..#.
.#....####""".strip().split("\n")

day10(test2)

(33, (5, 8))

In [231]:
test3 = """
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##""".strip().split("\n")

day10(test3)

(210, (11, 13))

In [233]:
day10(inputs)

(214, (8, 16))

### Part 2

Take a naive approach: create ordered dict `{angle: list of coordinate point}` where the list is ordered by the distance from the source. Then consume the lists in round robin manner.

In [289]:
def coord_to_angle(target, source):
    """Get angle as range [0, 2*pi)
    """
    x, y = target
    x0, y0 = source
    angle = math.atan2(y - y0, x - x0) + math.pi / 2
    angle %= 2 * math.pi
    return angle


def get_sorted_dict(asteroid_locs, source):
    def _dist(p):
        x0, y0 = source
        x, y = p
        return math.hypot(x - x0, y - y0)
    
    d = collections.defaultdict(list)
    for p in asteroid_locs:
        if p == source:
            continue
        angle = coord_to_angle(p, source)
        d[angle].append(p)
    
    # sort each list
    for k in d:
        d[k] = sorted(d[k], key=_dist)
    
    # sort dictionary keys
    # Note that the order of dictionary entreies is fixed since python 3.6 or so
    d = {k: d[k] for k in sorted(d)}
    return d


# https://docs.python.org/3/library/itertools.html
def roundrobin(*iterables):
    "roundrobin('ABC', 'D', 'EF') --> A D E B F C"
    # Recipe credited to George Sakkis
    num_active = len(iterables)
    nexts = it.cycle(iter(it).__next__ for it in iterables)
    while num_active:
        try:
            for next in nexts:
                yield next()
        except StopIteration:
            # Remove the iterator we just exhausted from the cycle.
            num_active -= 1
            nexts = it.cycle(it.islice(nexts, num_active))

            
def day10_mod_iterator(grid, source):
    asteroid_locs = _get_asteroid_locs(grid)
    d = get_sorted_dict(asteroid_locs, source)
    iterable = roundrobin(*d.values())
    return iterable


def day10_mod(grid, source=(8, 16)):
    iterable = day10_mod_iterator(grid, source)
    for i, p in enumerate(iterable, 1):
        if i == 200:
            x, y = p
            break
    return 100 * x + y

In [290]:
test2 = """
.#..##.###...#######
##.############..##.
.#.######.########.#
.###.#######.####.#.
#####.##.#.##.###.##
..#####..#.#########
####################
#.####....###.#.#.##
##.#################
#####.##.###..####..
..######..##.#######
####.##.####...##..#
.#####..#.######.###
##...#.##########...
#.##########.#######
.####.#.###.###.#.##
....##.##.###..#####
.#.#.###########.###
#.#.#.#####.####.###
###.##.####.##.#..##
""".strip().split("\n")

iterable = day10_mod_iterator(test2, (11, 13))
for i, p in enumerate(iterable, 1):
    if i in {1, 2, 3, 10, 20, 50, 100, 199, 200, 201, 299}:
        print(f"{i}-th point: {p}")

1-th point: (11, 12)
2-th point: (12, 1)
3-th point: (12, 2)
10-th point: (12, 8)
20-th point: (16, 0)
50-th point: (16, 9)
100-th point: (10, 16)
199-th point: (9, 6)
200-th point: (8, 2)
201-th point: (10, 9)
299-th point: (11, 1)


In [291]:
day10_mod(test2, (11, 13))

802

In [292]:
day10_mod(inputs, (8, 16))

502